# Homework2: Autoregressive models & VAE

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt


from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

USE_CUDA = torch.cuda.is_available()

In [ ]:
from utils import train_model, plot_training_curves
from utils import show_samples, visualize_mnist_images, load_pickle
from utils import grid_preprocessing

In [ ]:
import wandb
wandb.login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Task 1: PixelCNN on MNIST (5pt)

[PixelCNN](https://arxiv.org/abs/1601.06759) model uses masked causal convoultions on images, we have discussed this model on lecture 2.

Here you have to train this model on MNIST images.

(see paper for details: https://arxiv.org/abs/1601.06759)

In [ ]:
# change the path to the file
train_data, test_data = load_pickle(
    os.path.join('drive', 'My Drive', 'DGM', 'homework_supplementary', 'mnist.pkl'),
    flatten=False
)

train_data = torch.tensor(train_data).long()
test_data = torch.tensor(test_data).long()

visualize_mnist_images(train_data.reshape(-1, 1, 28, 28), 'MNIST samples')

In [ ]:
class MaskedConv2d(nn.Conv2d):
    def __init__(self, mask_type, in_channels, out_channels, kernel_size=5):
        assert mask_type in ['A', 'B']
        super().__init__(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        self.register_buffer('mask', torch.zeros_like(self.weight))
        self.create_mask(mask_type)

    def forward(self, input):
        # ====
        # your code
        
        # ====

    def create_mask(self, mask_type):
        # ====
        # your code
        # do not forget about mask_type
        
        # ====


def test_masked_conv2d():
    layer = MaskedConv2d('A', 2, 2)
    assert np.allclose(layer.mask[:, :, 2, 2].numpy(), np.zeros((2, 2)))

    layer = MaskedConv2d('B', 2, 2)
    assert np.allclose(layer.mask[:, :, 2, 2].numpy(), np.ones((2, 2)))


test_masked_conv2d()

[Layer Normalization](https://arxiv.org/abs/1607.06450) helps to stabilize training process.

In [ ]:
class LayerNorm(nn.LayerNorm):
    def __init__(self, n_filters):
        super().__init__(n_filters)

    def forward(self, x):
        x = x.permute(0, 2, 3, 1).contiguous()
        x = super().forward(x)
        return x.permute(0, 3, 1, 2).contiguous()

In [ ]:
class PixelCNN(nn.Module):
    def __init__(
        self, 
        input_shape, 
        n_filters=64, 
        kernel_size=7, 
        n_layers=5, 
        use_layer_norm=True
    ):
      
        super().__init__()
        self.input_shape = input_shape

        # ====
        # your code
        # apply the sequence of MaskedConv2d -> LayerNorm -> ReLU
        # note that the first conv layer should be of type 'A'
        # the last layer should be MaskedConv2d
        # NOTE 1: padding must be (kernel_size - 1) // 2
        # NOTE 2: final output_dim in MaskedConv2d must be 2
        
        # ====

    def forward(self, x):
        batch_size = x.shape[0]
        out = (x.float() - 0.5) / 0.5
        out = self.net(out)
        return out.view(batch_size, 2, 1, *self.input_shape)

    def loss(self, x):
        # ====
        # your code
        
        # ====
        total_loss = ...
        return {'total_loss': total_loss}

    def sample(self, n, use_cuda=True):
        # read carefully the sampling process
        samples = torch.zeros(n, 1, *self.input_shape)
        if use_cuda:
            samples = samples.cuda()
            
        with torch.no_grad():
            for r in range(self.input_shape[0]):
                for c in range(self.input_shape[1]):
                    logits = self(samples)[:, :, :, r, c]
                    probs = F.softmax(logits, dim=1).squeeze(-1)
                    samples[:, 0, r, c] = torch.multinomial(probs, num_samples=1).squeeze(-1)
        return samples.permute(0, 2, 3, 1).cpu().numpy()

In [ ]:
wandb.init(project="your project name")
# ====
# your code
EPOCHS =      # < 10
BATCH_SIZE =  # any adequate value
LR =          # < 1e-2
N_LAYERS =    # 2 < _ < 5
N_FILTERS =   # 16 < _ < 64
# ====

model = PixelCNN(
    input_shape=(28, 28), 
    n_filters=N_FILTERS, 
    kernel_size=5, 
    n_layers=N_LAYERS, 
    use_layer_norm=True
)

loss = model.loss(torch.zeros(1, 1, 28, 28))
assert isinstance(loss, dict)
assert 'total_loss' in loss

train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE)
train_losses, test_losses = train_model(model, train_loader, test_loader, preprocess=grid_preprocessing,
                                        epochs=EPOCHS, lr=LR, use_tqdm=True, use_cuda=USE_CUDA)

assert test_losses['total_loss'][-1] < 0.11

In [ ]:
plot_training_curves(train_losses, test_losses)

In [ ]:
with torch.no_grad():
    samples = model.sample(25)
show_samples(samples, title='MNIST samples', preprocess=grid_preprocessing)

The PixelCNN model is a powerful model. The samples should have better quality compared to MADE model from HW1. But the model has drawbacks.

1. The model is sequential and sampling is really slow (it is a drawback of all AR models).

2. The receptive field of the model is not so large. Even if the model is well-trained, the samples do not have long-range history. To validate this let try to visualize the receptive field of the model. In order to do this we will use the following function. It is really helpful to understand this code.

In [ ]:
def plot_receptive_field(model, data, n_layers):
    out = model(data)
    x_center, y_center = data.shape[-2] // 2, data.shape[-1] // 2
    out[0, 0, 0, x_center, y_center].backward()
    grad = data.grad.detach().cpu().numpy()[0, 0]
    grad = np.abs(grad)
    binary_map = (grad > 1e-8).astype('float32')
    binary_map = np.stack([binary_map, binary_map, binary_map], axis=-1)
    binary_map[x_center, y_center] = [1, 0, 0]
    weighted_map = grad * (1 / grad.max())
    weighted_map = np.stack([weighted_map, weighted_map, weighted_map], axis=-1)
    weighted_map[x_center, y_center] = [1, 0, 0]

    fig, ax = plt.subplots(1,2, figsize=(10, 6))
    ax[0].imshow(weighted_map, vmin=0.0, vmax=1.0)
    ax[1].imshow(binary_map, vmin=0.0, vmax=1.0)

    ax[0].set_title(f"Weighted receptive field, PixelCNN {n_layers} layers")
    ax[1].set_title(f"Binary receptive field, PixelCNN {n_layers} layers")
    plt.show()

In [ ]:
x = torch.zeros(1, 1, 28, 28)

if USE_CUDA:
    x = x.cuda()
x.requires_grad = True

for n_layers in [1, 3, 5, 6]:
    model = PixelCNN(
        input_shape=(28, 28), 
        n_filters=32, 
        kernel_size=5, 
        n_layers=n_layers, 
        use_layer_norm=True
    )
    if USE_CUDA:
        model = model.cuda()
    plot_receptive_field(model, x, n_layers)
    x.grad.zero_()

You have to see that PixelCNN has strange blind spot in binary receptive field plot on the right side. This is a known issue of PixelCNN model. Please, try to understand why it happens. 

One way to solve this problem is a GatedPixelCNN model (see paper https://arxiv.org/pdf/1606.05328.pdf, if you are interested in).

## Task 2: Theory (3pt)

In Lecture 3, when deriving the ELBO gradient at the E-step, we encountered a problem with Monte Carlo estimation, since the distribution function depended on the differentiation parameters.
\begin{align*}
    \nabla_{\boldsymbol{\phi}} \mathcal{L} (\boldsymbol{\phi}, \boldsymbol{\theta}) &= \nabla_{\boldsymbol{\phi}} \int q(\mathbf{z} | \mathbf{x}, \boldsymbol{\phi}) \left[\log p(\mathbf{x}, \mathbf{z} | \boldsymbol{\theta}) - \log q(\mathbf{z}| \mathbf{x}, \boldsymbol{\phi}) \right] d \mathbf{z} \\
    & \neq  \int q(\mathbf{z} | \mathbf{x}, \boldsymbol{\phi}) \nabla_{\boldsymbol{\phi}} \left[\log p(\mathbf{x}, \mathbf{z} | \boldsymbol{\theta}) - \log q(\mathbf{z}| \mathbf{x}, \boldsymbol{\phi}) \right] d \mathbf{z} \\
\end{align*}
The Reparametrization trick allowed us to skip the gradient and get a Monte Carlo estimate. But there is another way that uses the so-called **log-derivative trick**:
$$
    \nabla_\xi  \log q(\eta| \xi) = \frac{\nabla_\xi q(\eta| \xi)}{q(\eta| \xi)}.
$$
* Get the Monte Carlo estimate of the gradient using the formula for the derivative of the logarithm.
* The resulting expression works significantly worse than the reparametrization trick. Namely, it has a huge variance. Try to describe the intuition why the evaluation has a high variance (you need to think about what order and sign the terms in the expression will have).

```
your solution
```

## Task 2: VAE on 2d data (5pt)

In this task you will implement simple VAE model for 2d gaussian distribution.

We will consider two cases: 2d univariate distribution and 2d multivariate distribution. The goal is to analyze the difference between these two cases and chosen VAE model.

Below you will find data generation function. Look carefully, do not change.

Below you will find the functions which visualizes 2d data and plots the obtained training curves.
Study them carefully but do not change them.

In [ ]:
def generate_2d_data(count, mode='univariate'):
    assert mode in ['univariate', 'multivariate']
    np.random.seed(42)
    mean = [[2.0, 3.0]]
    sigma = [[3.0, 1.0]]
    if mode == 'univariate':
        rotate = [
            [1.0, 0.0], 
            [0.0, 1.0]
        ]
    else:
        rotate = [
            [np.sqrt(2) / 2, np.sqrt(2) / 2], 
            [-np.sqrt(2) / 2, np.sqrt(2) / 2]
        ]
    data = mean + (np.random.randn(count, 2) * sigma).dot(rotate)
    data = data.astype('float32')
    split = int(0.7 * count)
    train_data, test_data = data[:split], data[split:]
    return train_data, test_data


def visualize_2d_data(train_data, test_data, train_labels=None, test_labels=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.set_title('train', fontsize=16)
    ax1.scatter(train_data[:, 0], train_data[:, 1], s=1, c=train_labels)
    ax1.tick_params(labelsize=16)
    ax2.set_title('test', fontsize=16)
    ax2.scatter(test_data[:, 0], test_data[:, 1], s=1, c=test_labels)
    ax2.tick_params(labelsize=16)
    plt.show()


def visualize_2d_samples(data, title):
    plt.figure(figsize=(5, 5))
    plt.scatter(data[:, 0], data[:, 1], s=1)
    plt.title(title, fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.show()


def get_latent_stats(model, test_data, use_cuda=True, batch_size=3000):
    batch = next(iter(data.DataLoader(test_data, batch_size=batch_size, shuffle=True)))
    if use_cuda:
        batch = batch.cuda()

    with torch.no_grad():
        mu_z, log_std_z = model(batch)[:2]
        
    mu_z = mu_z.cpu().numpy()
    std_z = log_std_z.exp().cpu().numpy()

    return mu_z, std_z

In [ ]:
COUNT = 15000

train_data, test_data = generate_2d_data(COUNT, mode='multivariate')
visualize_2d_data(train_data, test_data)

train_data, test_data = generate_2d_data(COUNT, mode='univariate')
visualize_2d_data(train_data, test_data)

The difference of these two cases is the form of covariance matrix.

In multivariate case the matrix is non-diagonal, in univariate case it is strictly diagonal. As you will see, our VAE model will have absolutely different results for these datasets.

Now it is time to define our model. In this task you will create VAE model on 2d data. Model will be designed as follows:

* the latent dimensionality is equal to 2, the same as the data dimensionality ($\mathbf{z} \in \mathbb{R}^2$, $\mathbf{x} \in \mathbb{R}^2$).
* prior distribution $p(\mathbf{z}) = \mathcal{N}(0, I)$.
* approximate variational distribution (or encoder) $q(\mathbf{z} | \mathbf{x}, \boldsymbol{\phi}) = \mathcal{N}(\boldsymbol{\mu}_{\boldsymbol{\phi}}(\mathbf{x}), \boldsymbol{\Sigma}_{\boldsymbol{\phi}}(\mathbf{x}))$. Here $\boldsymbol{\phi}$ denotes all parameters of the encoder neural network. 
* generative distribution (or decoder) $p(\mathbf{x} | \mathbf{z}, \boldsymbol{\theta}) = \mathcal{N}(\boldsymbol{\mu}_{\boldsymbol{\theta}}(\mathbf{z}), \boldsymbol{\Sigma}_{\boldsymbol{\theta}}(\mathbf{z}))$. Here $\boldsymbol{\theta}$ denotes all parameters of the decoder neural network. Please note, that here we will use continuous distribution at our variables.
* We will consider only diagonal covariance matrices $\boldsymbol{\Sigma}_{\boldsymbol{\phi}}(\mathbf{x})$, $\boldsymbol{\Sigma}_{\boldsymbol{\theta}}(\mathbf{z})$.

Model objective is ELBO:
$$
    \mathcal{L}(\boldsymbol{\phi}, \boldsymbol{\theta}) = \mathbb{E}_{q(\mathbf{z} | \mathbf{x}, \boldsymbol{\phi})} \log p(\mathbf{x} | \mathbf{z}, \boldsymbol{\theta}) - KL (q(\mathbf{z} | \mathbf{x}, \boldsymbol{\phi}) || p(\mathbf{z})).
$$

To make the expectation is independent of parameters $\boldsymbol{\phi}$, we will use reparametrization trick.

To calculate the loss, you should derive
- $\log p(\mathbf{x} | \mathbf{z}, \boldsymbol{\theta})$, note that generative distribution is $\mathcal{N}(\boldsymbol{\mu}_{\boldsymbol{\theta}}(\mathbf{z}), \boldsymbol{\Sigma}_{\boldsymbol{\theta}}(\mathbf{z}))$.
- KL between $\mathcal{N}(\boldsymbol{\mu}_{\boldsymbol{\phi}}(\mathbf{x}), \boldsymbol{\Sigma}_{\boldsymbol{\phi}}(\mathbf{x}))$ and $\mathcal{N}(0, I)$.

In [ ]:
def get_normal_KL(mean_1, log_std_1, mean_2=None, log_std_2=None):
    """
        This function should return the value of KL(p1 || p2),
        where p1 = Normal(mean_1, exp(log_std_1)), p2 = Normal(mean_2, exp(log_std_2) ** 2).
        If mean_2 and log_std_2 are None values, we will use standart normal distribution.
        Note that we consider the case of diagonal covariance matrix.
    """
    if mean_2 is None:
        mean_2 = torch.zeros_like(mean_1)
    if log_std_2 is None:
        log_std_2 = torch.zeros_like(log_std_1)
    # ====
    # your code
    
    # ====


def test_KL():
    assert np.isclose(get_normal_KL(torch.tensor(2), torch.tensor(3), torch.tensor(0), torch.tensor(0)).numpy(), 200.2144, rtol=1e-3)
    assert np.isclose(get_normal_KL(torch.tensor(2), torch.tensor(3), torch.tensor(4), torch.tensor(5)).numpy(), 1.50925, rtol=1e-3)
    assert np.allclose(get_normal_KL(torch.tensor((10, 10)), torch.tensor((2, 4)), torch.tensor((3, 5))).numpy(), [49.2990, 1498.479], rtol=1e-3)

test_KL()

In [ ]:
def get_normal_nll(x, mean, log_std):
    """
        This function should return the negative log likelihood log p(x),
        where p(x) = Normal(x | mean, exp(log_std) ** 2).
        Note that we consider the case of diagonal covariance matrix.
    """
    # ====
    # your code
    
    # ====


def test_NLL():
    assert np.isclose(get_normal_nll(torch.tensor(2), torch.tensor(2), torch.tensor(3)).numpy(), 3.9189, rtol=1e-3)
    assert np.isclose(get_normal_nll(torch.tensor(5), torch.tensor(-3), torch.tensor(6)).numpy(), 6.9191, rtol=1e-3)
    assert np.allclose(get_normal_nll(torch.tensor((10, 10)), torch.tensor((2, 4)), torch.tensor((3, 5))).numpy(), np.array([3.9982, 5.9197]), rtol=1e-3)

test_NLL()

In [ ]:
class FullyConnectedMLP(nn.Module):
    def __init__(self, input_shape, hiddens, output_shape):
        assert isinstance(hiddens, list)
        super().__init__()
        self.input_shape = (input_shape,)
        self.output_shape = (output_shape,)
        self.hiddens = hiddens

        # ====
        # your code 
        # Stack Dense layers with ReLU activation.
        # Note that you do not have to add relu after the last dense layer
        
        # ====

    def forward(self, x):
        # ====
        # your code
        # apply network that was defined in __init__ and return the output
        
        # ====


class VAE2d(nn.Module):
    def __init__(self, n_in, n_latent, enc_hidden_sizes, dec_hidden_sizes):
        assert isinstance(enc_hidden_sizes, list)
        assert isinstance(dec_hidden_sizes, list)
        super().__init__()
        self.n_latent = n_latent

        # ====
        # your code
        # define encoder and decoder networks
        # the encoder takes n_in elements, has enc_hidden_sizes neurons in hidden layers 
        # and outputs 2 * n_latent (n_latent for means, and n_latent for std)
        # the decoder takes n_latent elements, has dec_hidden_sizes neurons in hidden layers 
        # and outputs 2 * n_in (n_in for means, and n_in for std)
        
        # ====

    def prior(self, n, use_cuda=True):
        # ====
        # your code
        # return n samples from prior distribution (we use standart normal for prior)
        
        # ====
        z = ...
        if use_cuda:
            z = z.cuda()
        return z

    def forward(self, x):
        """
        Now you have to return from the model 
         - mu_z - means for variational distribution 
         - mu_x - means for generative distribution
         - log_std_z - logarithm of std for variational distribution
         - log_std_x - logarithm of std for generative distribution

        We use logarithm, since the std is always positive
        to get std we will exponentiate it to get rid of this constraint
        """
        # ====
        # your code

        # 1) mu_z, log_std_z are outputs from the encoder
        # 2) apply reparametrization trick to get z (input of decoder)
        #    (do not forget to use self.prior())
        # 3) mu_x, log_std_x are outputs from the decoder
        #    NOTE: it is usefull: [mu, log_std = decoder(input).chunk(2, dim=1)]
        
        # ====
        return mu_z, log_std_z, mu_x, log_std_x

    def loss(self, x):
        mu_z, log_std_z, mu_x, log_std_x = self(x)
        # ====
        # your code

        # 1) apply model to get mu_z, log_std_z, mu_x, log_std_x
        # 2) compute reconstruction loss using get_normal_nll (it is the first term in ELBO)
        # 3) compute KL loss using get_normal_KL (it is the second term in ELBO)
        
        # ====

        return {
            'elbo_loss': recon_loss + kl_loss, 
            'recon_loss': recon_loss,
            'kl_loss': kl_loss
        }

    def sample(self, n, noise=True, use_cuda=True):
        """
        to sample from VAE model you have to sample from prior
        and then apply decoder to prior samples.
        parameter noise indicates whether to sample from decoder
        or just use means of generative distribution as samples
        """
        with torch.no_grad():
            z = self.prior(n, use_cuda)

            # ====
            # your code
            
            # 1) apply decoder
            # 2) sample from the decoder distribution if noise=True
            # ====
            

        return x.cpu().numpy()

In [ ]:
# ====
# your code
# choose these parameters (2 hidden layers with number of neurons less than 10 would be enough for encoder and decoder)
ENC_HIDDEN_SIZES = 
DEC_HIDDEN_SIZES = 
BATCH_SIZE =        # any adequate value
EPOCHS =            # < 8
LR =                # < 1e-2
# ==== 

COUNT = 10000
train_data, test_data = generate_2d_data(COUNT, mode='multivariate')
visualize_2d_data(train_data, test_data)

In [ ]:

model = VAE2d(2, 2, ENC_HIDDEN_SIZES, DEC_HIDDEN_SIZES).cuda()
train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE)

train_losses, test_losses = train_model(
    model, train_loader, test_loader, epochs=EPOCHS, lr=LR, use_cuda=use_cuda, use_tqdm=use_tqdm, loss_key='elbo_loss'
)

In [ ]:
samples_noise = model.sample(3000, noise=True)
samples_nonoise = model.sample(3000, noise=False)

for key, value in test_losses.items():
    print('{}: {:.4f}'.format(key, value[-1]))

plot_training_curves(train_losses, test_losses)
visualize_2d_samples(samples_noise, title='Samples with Decoder Noise')
visualize_2d_samples(samples_nonoise, title='Samples without Decoder Noise')

In [ ]:
# just look at these numbers and read comments after this task
mu_z, std_z = get_latent_stats(model, test_data)

print('mu_z = ', mu_z.mean(axis=0), '+-', mu_z.std(axis=0))
print('std_z = ', std_z.mean(axis=0), '+-', std_z.std(axis=0))

In [ ]:
train_data, test_data = generate_2d_data(COUNT, mode='univariate')
visualize_2d_data(train_data, test_data)

model = VAE2d(2, 2, ENC_HIDDEN_SIZES, DEC_HIDDEN_SIZES).cuda()

In [ ]:
wandb.init(project="your project name")
train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE)

train_losses, test_losses = train_model(
    model, train_loader, test_loader, epochs=EPOCHS, lr=LR, use_cuda=use_cuda, use_tqdm=use_tqdm, loss_key='elbo_loss'
)

In [ ]:
samples_noise = model.sample(3000, noise=True)
samples_nonoise = model.sample(3000, noise=False)

for key, value in test_losses.items():
    print('{}: {:.4f}'.format(key, value[-1]))

plot_training_curves(train_losses, test_losses)
visualize_2d_samples(samples_noise, title='Samples with Decoder Noise')
visualize_2d_samples(samples_nonoise, title='Samples without Decoder Noise')

In [ ]:
# just look at these numbers and read comments after this task
mu_z, std_z = get_latent_stats(model, test_data)

print('mu_z = ', mu_z.mean(axis=0), '+-', mu_z.std(axis=0))
print('std_z = ', std_z.mean(axis=0), '+-', std_z.std(axis=0))

After training the model on these 2 datasets, have a look at "Samples without Decoder Noise" figures. These figures show the means of the generative distribution. In the case of multivariate gaussian, the means are perfectly aligned with data distribution. 
Otherwise, in the univariate gaussian case you have to see strange figure. This happens due to so called "posterior collapse" (we will discuss it on the one of our lecture).


Our posterior distribution $p(\mathbf{x} | \mathbf{z}, \boldsymbol{\theta}) = \mathcal{N}(\boldsymbol{\mu}_{\boldsymbol{\theta}}(\mathbf{z}), \boldsymbol{\Sigma}_{\boldsymbol{\theta}}(\mathbf{z}))$ is a univariate (covariance matrix is diagonal). Thus, to model univariate data distribution (second case) the model ignores latent code, cause the model fits the distribution without any information from latent space.

If the decoder ignores latent code, the second term in ELBO (KL) could be low (variational distribution, which is given by encoder model, is close to prior distribution for each datapoint). In the training curves you have to see that KL loss behaves differently in these two cases.

The mean and std of variational distribution also proves this concept. For the second case you have to see that mean is almost zero and std is almost one.

It is a real problem for generative models and we will discuss later how to overcome it.